In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

### Above is FIX for Unicode Error in Colab

01. How to run Mistral-7B-Instruct with Langchain and  BitsAndBytes
02. Simple RAG with Langchain / ChromaDB and  Mistral-7B-Instruct
03. Mistral-7B-Instruct with GGUF Format

In [ ]:
import shutil, os, subprocess
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/YouTube/')

Mounted at /content/drive


In [ ]:
# Read JSON file
import json
with open('myconfig.json') as data_file:
  myconfig = json.load(data_file)
print(myconfig.keys())

dict_keys(['OPEN_API_KEY', 'DB_HOST', 'DB_PORT', 'DB_USER', 'DB_PASSWORD', 'HF_TOKEN'])


In [1]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q -U einops
!pip install -q -U safetensors
!pip install -q -U torch
!pip install -q -U xformers
!pip install -q -U langchain
!pip install -q -U ctransformers[cuda]
!pip install chromadb
!pip install sentence-transformers

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.8.0 requires cubinlinker, which is not installed.
cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 23.8.0 requires ptxcompiler, which is not installed.
cuml 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
keras-cv 0.8.2 requires keras-core, which is not installed.
keras-nlp 0.8.1 requires keras-core, which is not installed.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.8 which is incompatible.
apache-beam 2.46.0 requires numpy<1.25.0,>=1.14.3, but you have numpy 1.26.4 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 11.0.0 which is incompatible.
cudf

01 : Running With LangChain

In [4]:
import torch
from transformers import BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

In [2]:
model_id = "mistralai/Mistral-7B-Instruct-v0.1"

In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
model_4bit = AutoModelForCausalLM.from_pretrained( model_id, device_map="auto",quantization_config=quantization_config, )
tokenizer = AutoTokenizer.from_pretrained(model_id)

2024-03-09 08:19:11.067894: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-09 08:19:11.068000: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-09 08:19:11.202065: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [6]:
pipeline = pipeline(
        "text-generation",
        model=model_4bit,
        tokenizer=tokenizer,
        use_cache=True,
        device_map="auto",
        max_length=5000,
        do_sample=True,
        top_k=5,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
)

In [7]:
from langchain import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain
llm = HuggingFacePipeline(pipeline=pipeline)

In [8]:
def generate_plot_syn(Transcript):
  if len(Transcript)>4500:
    return Transcript
  prompt = PromptTemplate(template=template, input_variables=["Transcript"])
  llm_chain = LLMChain(prompt=prompt, llm=llm)
  response = llm_chain.run({"Transcript":Transcript})
  return response


In [10]:
%%time
template = """<s>[INST] You are a helpful, respectful and honest assistant.
For the given movie Trailer Transcript, I want you to identify the plot synopsis of the movie and
generate the output as a text.
Generate a plot synopsis based on the Transcript below :
{Transcript}
 [/INST] </s>
"""

Transcript = """ what is this that I found in your bedroom ma it is a lamp and this is where you turn it on now here they've always had high aspirations got blunt got weave now they're taking a shot at higher education if I study huh take the test huh get high scores congratulations these scores are going to get you into any college in the country join us at reparations Technical Institute and learn hatred for the white devil in a relaxed campus atmosphere next taking a vow of celibacy next Prophet music artists and the women [Â __Â ] I gotta roll up that joint dog this institution has graduated countless Fortune 500 CEOs and six presidents Method Man is Silas are we in the right place oh [Â __Â ] we got a black man for a teacher Redman is Jamal this is crew what you mean the name's Jamal and I'll mess your crew up it's rowing Jimmy Jam NWA okay what you know about that I love [Â __Â ] with attitudes just be cool and you might have some fun this semester we have we have a stone party Cyrus I know what you're trying to do okay you're trying to figure give me some Power this winter for warning gentlemen see I'm looking for the Ecstasy for the pot for those shrooms academics all about good grades is that right Mr King yes sir Dean Kane sir and respect my bad did I spoil the moment are going up in smoke how high look at a fair woman study I love [Â __Â ] The movie is a cult"""
prompt = PromptTemplate(template=template, input_variables=["Transcript"])
llm_chain = LLMChain(prompt=prompt, llm=llm)
response = llm_chain.run({"Transcript":Transcript})
response

/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


CPU times: user 15.8 s, sys: 351 ms, total: 16.1 s
Wall time: 17.3 s


"\nThe movie revolves around a group of students at Reparations Technical Institute who are on a mission to learn hatred for the white devil. The students, including Silas, Jamal, and Jimmy Jam, are excited to be at the institution, which has a relaxed campus atmosphere and a reputation for producing successful CEOs and presidents. They are eager to study and learn from their black mentors, Method Man and Redman. The group is determined to achieve their academic goals and have a great time, even if it means rolling up joints and smoking pot. However, things take a dangerous turn when the students become obsessed with the idea of achieving power through any means necessary, including violence. They start to engage in criminal activities and their actions have serious consequences. As the movie progresses, the students' descent into a cult-like mentality leads to tragic consequences."

In [ ]:
%%time
template = """<s>[INST] You are a helpful, respectful and honest assistant.
For the given movie Trailer Transcript, I want you to identify the plot synopsis of the movie and
generate the output as a text.
Generate a plot synopsis based on the Transcript below :
{Transcript}
 [/INST] </s>
"""

Transcript = """ what is this that I found in your bedroom ma it is a lamp and this is where you turn it on now here they've always had high aspirations got blunt got weave now they're taking a shot at higher education if I study huh take the test huh get high scores congratulations these scores are going to get you into any college in the country join us at reparations Technical Institute and learn hatred for the white devil in a relaxed campus atmosphere next taking a vow of celibacy next Prophet music artists and the women [Â __Â ] I gotta roll up that joint dog this institution has graduated countless Fortune 500 CEOs and six presidents Method Man is Silas are we in the right place oh [Â __Â ] we got a black man for a teacher Redman is Jamal this is crew what you mean the name's Jamal and I'll mess your crew up it's rowing Jimmy Jam NWA okay what you know about that I love [Â __Â ] with attitudes just be cool and you might have some fun this semester we have we have a stone party Cyrus I know what you're trying to do okay you're trying to figure give me some Power this winter for warning gentlemen see I'm looking for the Ecstasy for the pot for those shrooms academics all about good grades is that right Mr King yes sir Dean Kane sir and respect my bad did I spoil the moment are going up in smoke how high look at a fair woman study I love [Â __Â ] The movie is a cult"""
prompt = PromptTemplate(template=template, input_variables=["Transcript"])
llm_chain = LLMChain(prompt=prompt, llm=llm)
response = llm_chain.run({"Transcript":Transcript})
response

In [11]:
import pandas as pd

In [14]:
test_data=pd.read_csv("/kaggle/input/transcription-merged/transcription_merged.csv")

In [15]:
test_data.head()

,Unnamed: 0,imdb_id,plot_synopsis
0,0,tt0278488,what is this that I found in your bedroom ma i...
1,1,tt0327850,"The movie is a cult, comedy, entertaining, vi..."
2,2,tt1758795,please welcome to the stage president of mathl...
3,3,tt4978274,we're all on a journey it seems somewhere alon...
4,4,tt2322441,"The movie is a pornographic, stupid"


In [16]:
test_data['plot_sumarry'] = test_data['plot_synopsis'].apply(generate_plot_syn)

/opt/conda/lib/python3.10/site-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentiall

In [17]:
test_data.head()

,Unnamed: 0,imdb_id,plot_synopsis,plot_sumarry
0,0,tt0278488,what is this that I found in your bedroom ma i...,\nThe movie is a satirical comedy about a grou...
1,1,tt0327850,"The movie is a cult, comedy, entertaining, vi...","The movie is a cult, comedy, and entertaining ..."
2,2,tt1758795,please welcome to the stage president of mathl...,Please allow me to introduce the plot synopsis...
3,3,tt4978274,we're all on a journey it seems somewhere alon...,"Based on the given movie trailer transcript, i..."
4,4,tt2322441,"The movie is a pornographic, stupid",The plot synopsis of this movie is unclear and...


In [18]:
test_data_sub=test_data[['imdb_id','plot_sumarry']]

In [25]:
test_data_sub= test_data_sub.rename(columns={'plot_sumarry': 'plot_synopsis'})

In [26]:
test_data_sub

,imdb_id,plot_synopsis
0,tt0278488,\nThe movie is a satirical comedy about a grou...
1,tt0327850,"The movie is a cult, comedy, and entertaining ..."
2,tt1758795,Please allow me to introduce the plot synopsis...
3,tt4978274,"Based on the given movie trailer transcript, i..."
4,tt2322441,The plot synopsis of this movie is unclear and...
...,...,...
704,tt2474972,The movie appears to be a suspenseful and viol...
705,tt0434409,\nThe movie is a suspenseful neo-noir murder m...
706,tt0450314,\nThe plot of this movie revolves around a neo...
707,tt1821478,"\nBased on the given movie trailer transcript,..."


In [27]:
test_data_sub.to_csv("Sub_final1.csv")